In [137]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat

# Para b_media

## Intervalo entorno a la media

Importar datos del problema

In [138]:
data = loadmat("Datos_Manufacturing_media.mat")
A_media = data['A_media']
A_hat_media = data['A_hat_media']
b_media = data['b_media'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [139]:
n_vars = A_media.shape[1]
x = cp.Variable(n_vars)
constraints = [A_media @ x <= b_media, x >= 0]
objective = cp.Maximize(c @ x)  # ← usar directamente Maximize
history = []
iteration = 0

while iteration < max_iter:
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if x.value is None or problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"\nProblema infactible o mal definido en la iteración {iteration}")
        break

    x_val = x.value
    f_val = float(c @ x_val)
    history.append(np.copy(x_val))

    print(f"\nIteración {iteration}")
    print(f"x* = [{', '.join(f'{xi:.4f}' for xi in x_val)}]")
    print(f"f(x*) = {f_val:.4f}")

    violated = False

    for i in range(len(A)):
        z = A_hat_media[i] * np.abs(x_val)
        top_idx = np.argsort(-z)[:Gamma]

        a_bar = np.copy(A[i])
        a_bar[top_idx] += A_hat_media[i][top_idx] * np.sign(x_val[top_idx])
        lhs = a_bar @ x_val

        if lhs > b_media[i] + tolerance:
            violated = True
            constraints.append(a_bar @ x <= b_media[i])

            terms = [f"{a_bar[j]:.4f}·x{j+1}" for j in range(n_vars)]
            expr_str = " + ".join(terms)

            print(f"- Se añade corte en restricción {i}:")
            print(f"   {expr_str} ≤ {b_media[i]:.4f}")
            print(f"   Valor actual: {lhs:.4f} > {b_media[i]:.4f}")
        else:
            print(f"Restricción {i} verificada: {lhs:.4f} ≤ {b_media[i]:.4f}")

    if not violated:
        print("Solución robusta encontrada.")
        break

    iteration += 1

# --- Resultado final ---
print("\nResultado final:")
if x.value is not None:
    print("x robusto =", x.value)
    print("Costo objetivo =", c @ x.value)
else:
    print("No se encontró solución robusta.")


Iteración 0
x* = [0.0000, 0.0000, 0.0000, 0.0000, 23.1523]
f(x*) = 23.1523
- Se añade corte en restricción 0:
   119.0841·x1 + 74.4444·x2 + 98.3784·x3 + 72.8288·x4 + 92.6937·x5 ≤ 1625.5000
   Valor actual: 2146.0738 > 1625.5000
- Se añade corte en restricción 1:
   2.5565·x1 + 4.6209·x2 + 3.1984·x3 + 3.5658·x4 + 3.6319·x5 ≤ 70.4600
   Valor actual: 84.0860 > 70.4600
- Se añade corte en restricción 2:
   12.4796·x1 + 6.3234·x2 + 8.0759·x3 + 7.2580·x4 + 9.0544·x5 ≤ 203.6700
   Valor actual: 209.6293 > 203.6700

Iteración 1
x* = [2.7874, 0.0000, 0.0000, 17.7617, 0.0000]
f(x*) = 20.5491
- Se añade corte en restricción 0:
   119.0841·x1 + 64.4444·x2 + 105.3784·x3 + 81.8288·x4 + 87.6937·x5 ≤ 1625.5000
   Valor actual: 1785.3551 > 1625.5000
- Se añade corte en restricción 1:
   2.5565·x1 + 4.3209·x2 + 3.8984·x3 + 3.8158·x4 + 3.5319·x5 ≤ 70.4600
   Valor actual: 74.9004 > 70.4600
Restricción 2 verificada: 181.4626 ≤ 203.6700

Iteración 2
x* = [0.0000, 0.7454, 0.0000, 7.1271, 11.3379]
f(x*) = 

# Intervalo [min max]

In [140]:
data = loadmat("Datos_Manufacturing_MinMax.mat")
A_MinMax = data['A_MinMax']
A_hat_MinMax = data['A_hat_MinMax']
b_media = data['b_media'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [141]:
n_vars = A_MinMax.shape[1]
x = cp.Variable(n_vars)
constraints = [A_MinMax @ x <= b_media, x >= 0]
objective = cp.Maximize(c @ x)  # ← usar directamente Maximize
history = []
iteration = 0

while iteration < max_iter:
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if x.value is None or problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"\n❌ Problema infactible o mal definido en la iteración {iteration}")
        break

    x_val = x.value
    f_val = float(c @ x_val)
    history.append(np.copy(x_val))

    print(f"\nIteración {iteration}")
    print(f"x* = [{', '.join(f'{xi:.4f}' for xi in x_val)}]")
    print(f"f(x*) = {f_val:.4f}")

    violated = False

    for i in range(len(A)):
        z = A_hat_MinMax[i] * np.abs(x_val)
        top_idx = np.argsort(-z)[:Gamma]

        a_bar = np.copy(A[i])
        a_bar[top_idx] += A_hat_MinMax[i][top_idx] * np.sign(x_val[top_idx])
        lhs = a_bar @ x_val

        if lhs > b_media[i] + tolerance:
            violated = True
            constraints.append(a_bar @ x <= b_media[i])

            terms = [f"{a_bar[j]:.4f}·x{j+1}" for j in range(n_vars)]
            expr_str = " + ".join(terms)

            print(f"- Se añade corte en restricción {i}:")
            print(f"   {expr_str} ≤ {b_media[i]:.4f}")
            print(f"   Valor actual: {lhs:.4f} > {b_media[i]:.4f}")
        else:
            print(f"Restricción {i} verificada: {lhs:.4f} ≤ {b_media[i]:.4f}")

    if not violated:
        print("Solución robusta encontrada.")
        break

    iteration += 1

# --- Resultado final ---
print("\nResultado final:")
if x.value is not None:
    print("x robusto =", x.value)
    print("Costo objetivo =", c @ x.value)
else:
    print("No se encontró solución robusta.")


Iteración 0
x* = [6.5500, 0.0000, 16.6714, 0.0000, 0.0000]
f(x*) = 23.2214
- Se añade corte en restricción 0:
   154.0841·x1 + 64.4444·x2 + 148.3784·x3 + 72.8288·x4 + 137.1937·x5 ≤ 1625.5000
   Valor actual: 3482.9302 > 1625.5000
- Se añade corte en restricción 1:
   4.0215·x1 + 4.3209·x2 + 5.1784·x3 + 3.5658·x4 + 5.5219·x5 ≤ 70.4600
   Valor actual: 112.6717 > 70.4600
- Se añade corte en restricción 2:
   17.9446·x1 + 5.3234·x2 + 14.4409·x3 + 7.2580·x4 + 14.7594·x5 ≤ 203.6700
   Valor actual: 358.2883 > 203.6700

Iteración 1
x* = [0.0000, 0.0000, 0.0000, 19.7601, 0.0000]
f(x*) = 19.7601
- Se añade corte en restricción 0:
   104.0841·x1 + 113.9444·x2 + 148.3784·x3 + 123.3288·x4 + 87.6937·x5 ≤ 1625.5000
   Valor actual: 2436.9939 > 1625.5000
- Se añade corte en restricción 1:
   2.0565·x1 + 6.3109·x2 + 5.1784·x3 + 5.5158·x4 + 3.5319·x5 ≤ 70.4600
   Valor actual: 108.9923 > 70.4600
- Se añade corte en restricción 2:
   11.4796·x1 + 11.7684·x2 + 14.4409·x3 + 13.6680·x4 + 8.5544·x5 ≤ 203.

# Para b_max

## Intervalo entorno a la media

Importar datos del problema

In [142]:
b_max = data['b_max'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [143]:
n_vars = A_media.shape[1]
x = cp.Variable(n_vars)
constraints = [A_media @ x <= b_max, x >= 0]
objective = cp.Maximize(c @ x)  # ← usar directamente Maximize
history = []
iteration = 0

while iteration < max_iter:
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if x.value is None or problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"\nProblema infactible o mal definido en la iteración {iteration}")
        break

    x_val = x.value
    f_val = float(c @ x_val)
    history.append(np.copy(x_val))

    print(f"\n Iteración {iteration}")
    print(f"x* = [{', '.join(f'{xi:.4f}' for xi in x_val)}]")
    print(f"f(x*) = {f_val:.4f}")

    violated = False

    for i in range(len(A)):
        z = A_hat_media[i] * np.abs(x_val)
        top_idx = np.argsort(-z)[:Gamma]

        a_bar = np.copy(A[i])
        a_bar[top_idx] += A_hat_media[i][top_idx] * np.sign(x_val[top_idx])
        lhs = a_bar @ x_val

        if lhs > b_max[i] + tolerance:
            violated = True
            constraints.append(a_bar @ x <= b_max[i])

            terms = [f"{a_bar[j]:.4f}·x{j+1}" for j in range(n_vars)]
            expr_str = " + ".join(terms)

            print(f"-Se añade corte en restricción {i}:")
            print(f"   {expr_str} ≤ {b_max[i]:.4f}")
            print(f"   Valor actual: {lhs:.4f} > {b_max[i]:.4f}")
        else:
            print(f"Restricción {i} verificada: {lhs:.4f} ≤ {b_max[i]:.4f}")

    if not violated:
        print("Solución robusta encontrada.")
        break

    iteration += 1

# --- Resultado final ---
print("\nResultado final:")
if x.value is not None:
    print("x robusto =", x.value)
    print("Costo objetivo =", c @ x.value)
else:
    print("No se encontró solución robusta.")


 Iteración 0
x* = [0.0000, 0.0000, 0.0000, 12.2710, 18.7822]
f(x*) = 31.0532
-Se añade corte en restricción 0:
   104.0841·x1 + 74.4444·x2 + 98.3784·x3 + 81.8288·x4 + 92.6937·x5 ≤ 2192.0000
   Valor actual: 2745.1087 > 2192.0000
-Se añade corte en restricción 1:
   2.0565·x1 + 4.6209·x2 + 3.1984·x3 + 3.8158·x4 + 3.6319·x5 ≤ 93.5200
   Valor actual: 115.0374 > 93.5200
-Se añade corte en restricción 2:
   11.4796·x1 + 6.3234·x2 + 8.0759·x3 + 8.2580·x4 + 9.0544·x5 ≤ 261.5800
   Valor actual: 271.3946 > 261.5800

 Iteración 1
x* = [3.1087, 0.0000, 0.0000, 22.8334, 0.0000]
f(x*) = 25.9421
-Se añade corte en restricción 0:
   119.0841·x1 + 64.4444·x2 + 105.3784·x3 + 81.8288·x4 + 87.6937·x5 ≤ 2192.0000
   Valor actual: 2238.6309 > 2192.0000
-Se añade corte en restricción 1:
   2.5565·x1 + 4.3209·x2 + 3.8984·x3 + 3.8158·x4 + 3.5319·x5 ≤ 93.5200
   Valor actual: 95.0744 > 93.5200
Restricción 2 verificada: 227.3549 ≤ 261.5800

 Iteración 2
x* = [8.5011, 12.7516, 0.0000, 4.3737, 0.0000]
f(x*) = 

# Intervalo [min max]

In [144]:
b_max = data['b_max'].flatten()

c = np.array([1.0,1.0,1.0,1.0,1.0]) 
Gamma = 3
tolerance = 1e-6
max_iter = 30

In [145]:
n_vars = A_MinMax.shape[1]
x = cp.Variable(n_vars)
constraints = [A_MinMax @ x <= b_max, x >= 0]
objective = cp.Maximize(c @ x)  # ← usar directamente Maximize
history = []
iteration = 0

while iteration < max_iter:
    problem = cp.Problem(objective, constraints)
    problem.solve()

    if x.value is None or problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"\nProblema infactible o mal definido en la iteración {iteration}")
        break

    x_val = x.value
    f_val = float(c @ x_val)
    history.append(np.copy(x_val))

    print(f"\nIteración {iteration}")
    print(f"x* = [{', '.join(f'{xi:.4f}' for xi in x_val)}]")
    print(f"f(x*) = {f_val:.4f}")

    violated = False

    for i in range(len(A)):
        z = A_hat_MinMax[i] * np.abs(x_val)
        top_idx = np.argsort(-z)[:Gamma]

        a_bar = np.copy(A[i])
        a_bar[top_idx] += A_hat_MinMax[i][top_idx] * np.sign(x_val[top_idx])
        lhs = a_bar @ x_val

        if lhs > b_max[i] + tolerance:
            violated = True
            constraints.append(a_bar @ x <= b_max[i])

            terms = [f"{a_bar[j]:.4f}·x{j+1}" for j in range(n_vars)]
            expr_str = " + ".join(terms)

            print(f"- Se añade corte en restricción {i}:")
            print(f"   {expr_str} ≤ {b_max[i]:.4f}")
            print(f"   Valor actual: {lhs:.4f} > {b_max[i]:.4f}")
        else:
            print(f"Restricción {i} verificada: {lhs:.4f} ≤ {b_max[i]:.4f}")

    if not violated:
        print("Solución robusta encontrada.")
        break

    iteration += 1

# --- Resultado final ---
print("\nResultado final:")
if x.value is not None:
    print("x robusto =", x.value)
    print("Costo objetivo =", c @ x.value)
else:
    print("No se encontró solución robusta.")


Iteración 0
x* = [0.0000, 0.0000, 18.3565, 0.0000, 12.8353]
f(x*) = 31.1917
- Se añade corte en restricción 0:
   154.0841·x1 + 64.4444·x2 + 148.3784·x3 + 72.8288·x4 + 137.1937·x5 ≤ 2192.0000
   Valor actual: 4484.6209 > 2192.0000
- Se añade corte en restricción 1:
   4.0215·x1 + 4.3209·x2 + 5.1784·x3 + 3.5658·x4 + 5.5219·x5 ≤ 93.5200
   Valor actual: 165.9313 > 93.5200
- Se añade corte en restricción 2:
   17.9446·x1 + 5.3234·x2 + 14.4409·x3 + 7.2580·x4 + 14.7594·x5 ≤ 261.5800
   Valor actual: 454.5249 > 261.5800

Iteración 1
x* = [0.0000, 0.0000, 0.0000, 26.2272, 0.0000]
f(x*) = 26.2272
- Se añade corte en restricción 0:
   104.0841·x1 + 113.9444·x2 + 148.3784·x3 + 123.3288·x4 + 87.6937·x5 ≤ 2192.0000
   Valor actual: 3234.5681 > 2192.0000
- Se añade corte en restricción 1:
   2.0565·x1 + 6.3109·x2 + 5.1784·x3 + 5.5158·x4 + 3.5319·x5 ≤ 93.5200
   Valor actual: 144.6630 > 93.5200
- Se añade corte en restricción 2:
   11.4796·x1 + 11.7684·x2 + 14.4409·x3 + 13.6680·x4 + 8.5544·x5 ≤ 261